In [12]:
!pip install tensorflow

In [13]:
!pip install numpy scipy matplotlib scikit-learn

In [14]:
!pip install pandas

In [15]:
## import package
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, mean_squared_error
import pandas as pd

### Build up basic model


In [16]:
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Input(shape=input_shape))
    
    # 使用一维卷积层 (Conv1D) 处理一维序列数据
    model.add(layers.Conv1D(32, 3, activation='relu', padding='same'))
    model.add(layers.Conv1D(32, 3, activation='relu', padding='same'))
    model.add(layers.Conv1D(64, 3, activation='relu', padding='same'))
    model.add(layers.Conv1D(64, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Conv1D(128, 3, activation='relu', padding='same'))
    model.add(layers.Conv1D(128, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Conv1D(256, 3, activation='relu', padding='same'))
    model.add(layers.Conv1D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    
    # 展平层
    model.add(layers.Flatten())
    
    # 全连接层
    model.add(layers.Dense(512, activation='relu'))
    
    # 输出层，假设输出是分类问题
    model.add(layers.Dense(input_shape[0] * num_classes, activation='softmax'))
    model.add(layers.Reshape((input_shape[0], num_classes)))  # 将输出重塑为 (sequence_length, num_classes)
    
    return model

"""
input_shape = (100, 1)  # 示例输入形状
num_classes = 3  # 假设有3个类别
model = create_cnn_model(input_shape, num_classes)
model.summary()
"""

'\ninput_shape = (100, 1)  # 示例输入形状\nnum_classes = 3  # 假设有3个类别\nmodel = create_cnn_model(input_shape, num_classes)\nmodel.summary()\n'

## Train the model

In [17]:
def compile_model(model):
    # 使用Adam优化器，学习率为0.002
    optimizer = optimizers.Adam(learning_rate=0.002)
    # 编译模型，使用交叉熵损失函数和准确率作为指标
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [18]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_data(filename, num_classes):
    # 读取 Excel 文件中的所有工作表名称
    xls = pd.ExcelFile(filename)
    sheet_names = xls.sheet_names
    
    # 初始化用于存储所有工作表数据的列表
    X_data = []
    Y_data = []
    
    # 遍历每个工作表
    for sheet_name in sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        # 提取 X 列和 Y 列的数据
        X = df['X'].to_numpy().reshape(-1, 1)
        Y = df['Y'].to_numpy().reshape(-1, 1)
        
        # 将数据添加到列表中
        X_data.append(X)
        Y_data.append(Y)
    
    # 转换为 NumPy 数组并重塑为 (num_samples, sequence_length, num_features)
    X_data = np.array(X_data)
    X_data = X_data.reshape((X_data.shape[0], X_data.shape[1], 1))
    
    # 转换 Y_data 为 one-hot 编码并重塑为 (num_samples, sequence_length, num_classes)
    Y_data = np.array(Y_data)
    Y_data = to_categorical(Y_data, num_classes=num_classes)
    
    return X_data, Y_data


"""
# 从 train_data.xlsx 中加载数据
num_classes = 3  # 假设有3个类别
train_X, train_Y = load_data('train_data.xlsx', num_classes)

# 从 test_data.xlsx 中加载数据（如果有相同的结构）
test_X, test_Y = load_data('test_data.xlsx', num_classes)

# 打印一些数据以进行验证
print(f'Number of training samples: {train_X.shape[0]}')
print(f'First training sample X shape: {train_X[0].shape}')
print(f'First training sample Y shape: {train_Y[0].shape}')

print(f'Number of testing samples: {test_X.shape[0]}')
print(f'First testing sample X shape: {test_X[0].shape}')
print(f'First testing sample Y shape: {test_Y[0].shape}')
"""

"\n# 从 train_data.xlsx 中加载数据\nnum_classes = 3  # 假设有3个类别\ntrain_X, train_Y = load_data('train_data.xlsx', num_classes)\n\n# 从 test_data.xlsx 中加载数据（如果有相同的结构）\ntest_X, test_Y = load_data('test_data.xlsx', num_classes)\n\n# 打印一些数据以进行验证\nprint(f'Number of training samples: {train_X.shape[0]}')\nprint(f'First training sample X shape: {train_X[0].shape}')\nprint(f'First training sample Y shape: {train_Y[0].shape}')\n\nprint(f'Number of testing samples: {test_X.shape[0]}')\nprint(f'First testing sample X shape: {test_X[0].shape}')\nprint(f'First testing sample Y shape: {test_Y[0].shape}')\n"

In [19]:
def train_classification_model(model, X_train, y_train, X_val, y_val):
    # 使用所有数据同时进行训练，batch_size设置为训练集大小
    batch_size = len(X_train)
    # 训练模型，训练4000个epoch
    history = model.fit(X_train, y_train, epochs=4000, batch_size=batch_size, validation_data=(X_val, y_val))
    return history

In [20]:
def evaluate_classification_model(model, X_val, y_val):
    # 评估模型在验证集上的性能
    test_loss, test_acc = model.evaluate(X_val, y_val)
    print(f'Test accuracy: {test_acc}')
    
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    print(classification_report(y_val, y_pred_classes))

In [21]:
# 设置输入形状和类别数量
input_shape = (100,1)  # 示例输入形状
num_classes = 3  # 假设有3个类别

# 创建模型
model = create_cnn_model(input_shape, num_classes)
model.summary()

# 编译模型
compile_model(model)

# 加载和预处理数据

# 从 train_data.xlsx 中加载数据
num_classes = 3  
X_train, y_train = load_data('train_data.xlsx', num_classes)

# 从 test_data.xlsx 中加载数据（如果有相同的结构）
X_val, y_val = load_data('test_data.xlsx', num_classes)
# 训练模型
history = train_classification_model(model, X_train, y_train, X_val, y_val)

# 评估模型
evaluate_classification_model(model, X_val, y_val)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (None, 100, 32)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 100, 32)        │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 100, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 100, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_12 (Conv1D)              │ (None, 50, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 50, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 25, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 25, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 25, 256)        │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 12, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,573,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 300)            │       153,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 100, 3)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,118,476 (8.08 MB)

 Trainable params: 2,118,476 (8.08 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3507 - loss: 1.2222 - val_accuracy: 0.8551 - val_loss: 0.4944
Epoch 2/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step - accuracy: 0.8561 - loss: 0.4945 - val_accuracy: 0.9544 - val_loss: 0.1461
Epoch 3/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step - accuracy: 0.9500 - loss: 0.1669 - val_accuracy: 0.9581 - val_loss: 0.1631
Epoch 4/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step - accuracy: 0.9589 - loss: 0.1808 - val_accuracy: 0.9731 - val_loss: 0.1436
Epoch 5/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step - accuracy: 0.9690 - loss: 0.1765 - val_accuracy: 0.9744 - val_loss: 0.1016
Epoch 6/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step - accuracy: 0.9700 - loss: 0.1194 - val_accuracy: 0.9544 - val_loss: 0.0777
Epoch 7/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step - accuracy: 0.9533 - loss: 0.0820 - val_accuracy: 0.9717 - val_loss: 0.0688
Epoch 8/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step - accuracy: 0.9702 - loss: 0.0707 - val_accuracy: 0.967

KeyboardInterrupt: 

In [11]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'y_val shape: {y_val.shape}')

X_train shape: (1024, 100, 1)
y_train shape: (1024, 100, 3)
X_val shape: (1024, 100, 1)
y_val shape: (1024, 100, 3)
